In [2]:
!pip install python-docx==0.8.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184488 sha256=d02f9f1916766deb2d53d33b7869c246a58d0e06a46b054e941e91f466c34ca8
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [3]:
import os
from google.colab import userdata
import google.generativeai as genai
from typing import Tuple
from docx import Document
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
class AIArchitect:

    def __init__(self):
      try:
        api_key = userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        self.history = []
      except Exception as e:
        print("Error accessing API key from Colab secrets. Please ensure:")
        print("1. You've added a secret named 'GOOGLE_AI_KEY'")
        print("2. The secret contains a valid Google AI API key")
        raise
      self.history = []
      self.question = ""

    def ask(self) -> str:
      if self.question == "":
        self.question = "Please describe the system you want to build and its key functionalities."
        print(f"\n{self.question}")
        return input("Your response: ")
      else:
        print(f"\n{self.question}")
        return input("Your response (or enter 'g' to generate a design doc): ")

    def connect(self, answer: str) -> bool:

      if answer.lower() == 'g':
          return False

      self.history.append("AI Architect: " + self.question)
      self.history.append("User: " + answer)

      summary_prompt = f"""Analyze the conversation history and extract the top 5 most crucial design decisions made so far.

      Conversation History:
      {chr(10).join(self.history)}

      If fewer than 5 key decisions have been made, list only those available. Present them as a numbered list."""

      summary_response = self.model.generate_content(summary_prompt)

      # Generate a targeted follow-up question based on previous context
      question_prompt = f"""Based on the previous context:

      {chr(10).join(self.history)}

      Generate a concise, high-level question to further clarify the software architecture."""

      question_response = self.model.generate_content(question_prompt)
      self.question = question_response.text

      print("\n--- CURRENT DESIGN STATE ---\n")
      print(summary_response.text)
      print("----------------------------")

      return True

    def design(self) -> str:
        design_prompt = f"""Based on the following conversation history about the software architecture:

        {chr(10).join(self.history)}

        Create a professional, structured software architecture design document that includes:
        1. System Architecture Diagram
        2. Component Descriptions
        3. Communication and Interactions
        4. Technology Choices and Rationale
        5. Open Questions

        Be comprehensive and detailed."""

        design_response = self.model.generate_content(design_prompt)
        return design_response.text

    def save(self, design_doc: str, file_path: str):
        doc = Document()

        title = input("Document Title: ")
        doc.add_heading(title, 0)

        for line in design_doc.split('\n'):
          line = line.strip()
          if line.startswith('##'):
              # Add heading level 2
              doc.add_heading(line.lstrip('#').strip(), level=2)
          elif line.startswith('#'):
              # Add heading level 1
              doc.add_heading(line.lstrip('#').strip(), level=1)
          elif line.startswith('**') and line.endswith('**'):
              # Add bold paragraph
              p = doc.add_paragraph()
              p.add_run(line.strip('**')).bold = True
          # elif line.startswith('```') and line.endswith('```'):
          #     # Add code block
          #     doc.add_paragraph(line.strip('`'), style='Code')
          else:
              # Add regular paragraph
              doc.add_paragraph(line)

        doc.save(file_path)
        print(f"\n--- FINAL DESIGN DOCUMENT SAVED TO {file_path} ---\n")

    def run(self):

        while True:
            user_answer = self.ask()

            should_continue = self.connect(user_answer)

            if not should_continue:
                break

        final_design_doc = self.design()

        save_path = '/content/drive/My Drive/design_doc.docx'
        self.save(final_design_doc, save_path)

        return

In [47]:
ai_architect = AIArchitect()
ai_architect.run()


Please describe the system you want to build and its key functionalities.
Your response: I want to build a mobile app that scrapes TikTok to recommend the best local coffee shops.

--- CURRENT DESIGN STATE ---

Based on the provided conversation history, only one crucial design decision has been explicitly made:

1. **App Functionality:** The core functionality of the app will be to scrape TikTok data and recommend local coffee shops based on that data.  This encompasses several sub-decisions implicitly, such as data source (TikTok), target audience (coffee lovers), and output (recommendations).

----------------------------

What data from TikTok will be scraped (e.g., video captions, hashtags, geolocation), and how will this data be processed to identify and rank local coffee shops?

Your response (or enter 'g' to generate a design doc): Video transcript, comments, likes, and views will be scraped.

--- CURRENT DESIGN STATE ---

Based on the provided conversation history, here are t